In [1]:
import pandas as pd
import requests
import xmltodict
import json
from dataclasses import dataclass, asdict

In [2]:
@dataclass
class NumberSearch:
    number: int = ''
    version: str = '2.1'
    enumeration_type: any = 'NPI-2'
def get_results(search):
    url = 'https://npiregistry.cms.hhs.gov/api/?version=2.1'
    parameters = asdict(search)
    response =  requests.get(url, params=parameters)
    results = response.json()
    return results
def get_npi_info(results):
    result = results['results'][0]
    npi = result['number']
    for address in result['addresses']:
        if address['address_purpose']=='LOCATION':
            location_address_1 = address['address_1']
            location_address_2 = address['address_2']
    new_dict = {
                'NPI': npi,
                'LOCATION_ADDRESS_1': location_address_1,
                'LOCATION_ADDRESS_2': location_address_2,
            }
    return new_dict

In [3]:
filet = pd.read_csv('../../Data/Credentialling/Residents_Jun2019-Dec2019.csv')

In [4]:
npis = list(filet.drop_duplicates(['NPI_ORGANIZATION']).NPI_ORGANIZATION)

In [10]:
index+=1

In [15]:
get_results(NumberSearch(npi))

ConnectionError: ('Connection aborted.', OSError("(10054, 'WSAECONNRESET')"))

In [19]:
npi

1982017125

In [32]:
index = 0
for npi in nips[index:]:
    try:
        results = get_results(NumberSearch(npi))
        dict_list.append(get_npi_info(results))
        print(f'{index} is done')
        index+= 1
    except:
        index+= 1
        print(npi)
        nips.append(npi)
        continue

0 is done
1 is done
2 is done
3 is done
4 is done


In [25]:
l3 = [x for x in npis if x not in list(pd.DataFrame(dict_list).NPI)]

In [31]:
nips

[1902877715, 1588750616, 1902846306, 1629011986, 1376667170]

In [33]:
for thing in dict_list:
    if ' STE ' in thing['LOCATION_ADDRESS_1']:
        suite = thing['LOCATION_ADDRESS_1'].split('STE ')[1]
    elif 'SUITE' in thing['LOCATION_ADDRESS_1']:
        suite = thing['LOCATION_ADDRESS_1'].split('SUITE ')[1]
    elif ' STE ' in thing['LOCATION_ADDRESS_2']:
        suite = thing['LOCATION_ADDRESS_2'].split('STE ')[1]
    elif 'SUITE' in thing['LOCATION_ADDRESS_2']:
        suite = thing['LOCATION_ADDRESS_2'].split('SUITE ')[1]
    else:
        suite = 'None'
    thing['SUITE'] = suite

In [62]:
dict_list

[{'NPI': 1780686931,
  'LOCATION_ADDRESS_1': '1201 LANGHORNE-NEWTOWN RD',
  'LOCATION_ADDRESS_2': '',
  'SUITE': 'None'},
 {'NPI': 1386746592,
  'LOCATION_ADDRESS_1': '18 E LAUREL RD',
  'LOCATION_ADDRESS_2': '',
  'SUITE': 'None'},
 {'NPI': 1609417872,
  'LOCATION_ADDRESS_1': '4900 FRANKFORD AVE',
  'LOCATION_ADDRESS_2': '',
  'SUITE': 'None'},
 {'NPI': 1508971441,
  'LOCATION_ADDRESS_1': '130 S BRYN MAWR AVE',
  'LOCATION_ADDRESS_2': '',
  'SUITE': 'None'},
 {'NPI': 1811992084,
  'LOCATION_ADDRESS_1': '1200 OLD YORK RD',
  'LOCATION_ADDRESS_2': '',
  'SUITE': 'None'},
 {'NPI': 1649401142,
  'LOCATION_ADDRESS_1': '3400 SPRUCE ST',
  'LOCATION_ADDRESS_2': '1-SILVERSTEIN',
  'SUITE': 'None'},
 {'NPI': 1265664544,
  'LOCATION_ADDRESS_1': '454 ENTERPRISE DR',
  'LOCATION_ADDRESS_2': '',
  'SUITE': 'None'},
 {'NPI': 1831139955,
  'LOCATION_ADDRESS_1': '2701 BLAIR MILL RD',
  'LOCATION_ADDRESS_2': 'SUITE 35',
  'SUITE': '35'},
 {'NPI': 1942554308,
  'LOCATION_ADDRESS_1': '3900 WOODLAND AVE'

In [34]:
NEW = pd.DataFrame(dict_list).rename(columns={'NPI':'NPI_ORGANIZATION'})

In [35]:
datas = pd.merge(filet, NEW, on='NPI_ORGANIZATION')

In [38]:
datas= datas.rename(columns={'FULL_DT':'ORDER_DATE', 'NAME':'CUSTOMER_NAME'})

In [39]:
datas = datas[['ORDER_DATE','PRODUCT_DESC', 'CUSTOMER_NAME', 'CUSTOMER_CATEGORY_DESC', 'LOCATION_ADDRESS_1',
       'LOCATION_ADDRESS_2', 'SUITE', 'LOCATION_ZIP', 'LOCATION_CITY',
       'LOCATION_STATE','NPI_ORGANIZATION',
     'ME', 'LAST_NAME', 'FIRST_NAME',
       'MIDDLE_NAME', 'NPI_PHYSICIAN']]

In [40]:
def fix_me(me_list):
    nums = []
    for num in me_list:
        num = str(num)
        num = num.replace('.0', '')
        if len(num) == 10:
            num = '0' + num
        elif len(num) == 9:
            num = '00' + num
        elif len(num) == 8:
            num = '000' + num
        nums.append(num)
    return nums
datas['ME']=fix_me(list(datas['ME']))

In [41]:
def fix_zip(zip_list):
    nums = []
    for num in zip_list:
        num = str(num)
        num = num.replace('.0', '')
        if len(num) == 8:
            num = '0' + num
        elif len(num) == 7:
            num = '00' + num
        elif len(num) == 6:
            num = '000' + num
        elif len(num) == 4:
            num = '0' + num
        nums.append(num)
    return nums
datas['LOCATION_ZIP']=fix_zip(list(datas['LOCATION_ZIP']))

In [53]:
data_sample = datas.sample(2000)

In [54]:
data_sample.to_excel("../../Data/Credentialling/Resident_Sample_Jun2019-Dec2019.xlsx", index=False)

In [45]:
datas['ORDER_DATE'] = pd.to_datetime(datas.ORDER_DATE)

In [52]:
datas = datas.dropna(subset=['LAST_NAME'])